In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import cv2
from osgeo import gdal
import os

In [ ]:
# Read the image using OpenCV
img = cv2.imread("../data/20230215-SE2B-CGG-GBR-MS3-L3-RGB-preview.jpg", cv2.IMREAD_COLOR)

# Render the image using Matplotlib
plt.imshow(img)

In [ ]:
with rasterio.open("../data/DSM_TQ0075_P_12757_20230109_20230315.tif") as lidar:
    lidar_data = lidar.read(1)

# Plot the LIDAR data using Matplotlib
plt.imshow(lidar_data, cmap='plasma')
plt.colorbar()
plt.title('LIDAR Data')
plt.show()

In [ ]:
fig, ax = plt.subplots(figsize=(10, 10))

# Plot the original image
ax.imshow(img, extent=[0, img.shape[1], 0, img.shape[0]])

# Overlay the LIDAR data with transparency
ax.imshow(lidar_data, cmap='plasma', alpha=0.6, extent=[0, img.shape[1], 0, img.shape[0]])

ax.set_title('LIDAR Data Overlay on Original Image')
ax.axis('off')

plt.show()

In [ ]:
threshold_values = [5 * n for n in range(1, 10)]
fig, axes = plt.subplots(2, len(threshold_values) // 2, figsize=(20, 10))

for ax, threshold in zip(axes.flatten(), threshold_values):
    house_regions = lidar_data > threshold
    num_labels, labels_im = cv2.connectedComponents(house_regions.astype(np.uint8))
    
    ax.imshow(labels_im, cmap='nipy_spectral')
    ax.set_title(f'Threshold: {threshold}\nHouses: {num_labels - 1}')
    ax.axis('off')

plt.tight_layout()
plt.show()


In [ ]:
threshold_values = list(range(20, 40))
fig, axes = plt.subplots(4, 5, figsize=(20, 16))

for ax, threshold in zip(axes.flatten(), threshold_values):
    house_regions = lidar_data > threshold
    num_labels, labels_im = cv2.connectedComponents(house_regions.astype(np.uint8))
    
    ax.imshow(labels_im, cmap='nipy_spectral')
    ax.set_title(f'Threshold: {threshold}\nHouses: {num_labels - 1}')
    ax.axis('off')

plt.tight_layout()
plt.show()


In [ ]:
min_size = 50  # Minimum size of objects to be considered as houses
max_size = 1000  # Maximum size of objects to be considered as houses

threshold = 30
house_regions = lidar_data > threshold
num_labels, labels_im = cv2.connectedComponents(house_regions.astype(np.uint8))

# Filter out excessively large objects
filtered_labels_im = np.zeros_like(labels_im)
for label in range(1, num_labels):
    label_mask = labels_im == label
    label_size = np.sum(label_mask)
    if min_size <= label_size <= max_size:
        filtered_labels_im[label_mask] = label

fig, axes = plt.subplots(1, 2, figsize=(15, 10))

# Plot the original image
axes[0].imshow(img)
axes[0].set_title('Original Image')
axes[0].axis('off')

# Plot the filtered LIDAR data
axes[1].imshow(filtered_labels_im, cmap='nipy_spectral')
axes[1].set_title(f'Filtered LIDAR Data\nThreshold: {threshold}\nHouses: {np.max(filtered_labels_im)}')
axes[1].axis('off')

plt.tight_layout()
plt.show()

In [ ]:
min_size = 25  # Minimum size of objects to be considered as houses
max_size = 1500  # Maximum size of objects to be considered as houses

threshold = 30
house_regions = lidar_data > threshold
num_labels, labels_im = cv2.connectedComponents(house_regions.astype(np.uint8))

# Filter out excessively large objects
filtered_labels_im = np.zeros_like(labels_im)
for label in range(1, num_labels):
    label_mask = labels_im == label
    label_size = np.sum(label_mask)
    if min_size <= label_size <= max_size:
        filtered_labels_im[label_mask] = label

fig, axes = plt.subplots(1, 2, figsize=(15, 10))

# Plot the original image
axes[0].imshow(img)
axes[0].set_title('Original Image')
axes[0].axis('off')

# Plot the filtered LIDAR data
axes[1].imshow(filtered_labels_im, cmap='nipy_spectral')
axes[1].set_title(f'Filtered LIDAR Data\nThreshold: {threshold}\nHouses: {np.max(filtered_labels_im)}')
axes[1].axis('off')

plt.tight_layout()
plt.show()

In [ ]:
# Read the image using OpenCV
nrg = cv2.imread("../data/20230215-SE2B-CGG-GBR-MS3-L3-NRG-preview.jpg", cv2.IMREAD_COLOR)

# Render the image using Matplotlib
plt.imshow(cv2.cvtColor(nrg, cv2.COLOR_BGR2RGB))
plt.title('NRG Image')
plt.axis('off')
plt.show()

In [ ]:
# Convert the NRG image to RGB
nrg_img_rgb = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)

# Define the red color range in RGB
lower_red = np.array([100, 0, 0])
upper_red = np.array([255, 150, 150])

# Create a mask for red areas
red_mask = cv2.inRange(nrg_img_rgb, lower_red, upper_red)

# Resize the red mask to match the shape of the LIDAR data
red_mask_resized = cv2.resize(red_mask, (lidar_data.shape[1], lidar_data.shape[0]), interpolation=cv2.INTER_NEAREST)

# Subtract the red areas from the LIDAR data
lidar_data_subtracted = np.where(red_mask_resized == 255, np.nan, lidar_data)

# Plot the result
plt.imshow(lidar_data_subtracted, cmap='plasma')
plt.colorbar()
plt.title('LIDAR Data with Red Areas Subtracted')
plt.show()

In [ ]:
fig, axes = plt.subplots(6, 5, figsize=(20, 24))

for ax, ground_threshold in zip(axes.flatten(), range(1, 31)):
    # Create a mask for areas above the ground
    above_ground_mask = lidar_data > ground_threshold

    # Apply the mask to the LIDAR data
    above_ground_lidar_data = np.where(above_ground_mask, lidar_data, np.nan)

    # Plot the filtered LIDAR data
    ax.imshow(above_ground_lidar_data, cmap='plasma')
    ax.set_title(f'Ground Threshold: {ground_threshold}')
    ax.axis('off')

plt.tight_layout()
plt.show()

In [ ]:
ground_threshold = 22

# Create a mask for areas above the ground threshold
above_ground_mask = lidar_data > ground_threshold

# Apply the mask to the LIDAR data
above_ground_lidar_data = np.where(above_ground_mask, lidar_data, np.nan)

fig, ax = plt.subplots(figsize=(10, 10))

# Plot the original image
img = cv2.imread("/home/h00pyfr00d/Downloads/data/20230215-SE2B-CGG-GBR-MS3-L3-RGB-preview.jpg", cv2.IMREAD_COLOR)
ax.imshow(img, extent=[0, img.shape[1], 0, img.shape[0]])

# Overlay the LIDAR data with transparency
ax.imshow(above_ground_lidar_data, cmap='plasma', alpha=0.75, extent=[0, img.shape[1], 0, img.shape[0]])

ax.set_title('Ground Threshold 22 Overlay on Original Image')
ax.axis('off')

plt.show()

In [ ]:
# Convert the NRG image to RGB
nrg_img_rgb = cv2.cvtColor(nrg, cv2.COLOR_BGR2RGB)

# Define the red color range in RGB
lower_red = np.array([100, 0, 0])
upper_red = np.array([255, 150, 150])

# Create a mask for red areas
red_mask = cv2.inRange(nrg_img_rgb, lower_red, upper_red)

# Apply the mask to the original image
img_no_red = cv2.bitwise_and(img, img, mask=cv2.bitwise_not(red_mask))

fig, axes = plt.subplots(1, 2, figsize=(15, 10))

# Plot the original image
axes[0].imshow(cv2.cvtColor(img, cv2.COLOR_BGR2RGB))
axes[0].set_title('Original Image')
axes[0].axis('off')

# Plot the image with red areas removed
axes[1].imshow(cv2.cvtColor(img_no_red, cv2.COLOR_BGR2RGB))
axes[1].set_title('Image with Red Areas Removed')
axes[1].axis('off')

plt.tight_layout()
plt.show()

In [ ]:
fig, ax = plt.subplots(figsize=(10, 10))

# Plot the image with red areas removed
ax.imshow(cv2.cvtColor(img_no_red, cv2.COLOR_BGR2RGB), extent=[0, img_no_red.shape[1], 0, img_no_red.shape[0]])

# Overlay the filtered LIDAR data with transparency
ax.imshow(filtered_labels_im, cmap='nipy_spectral', alpha=0.6, extent=[0, img_no_red.shape[1], 0, img_no_red.shape[0]])

ax.set_title('Filtered LIDAR Data Overlay on Image with Red Areas Removed')
ax.axis('off')

plt.show()

In [ ]:
# Convert the NRG image to RGB
nrg_img_rgb = cv2.cvtColor(nrg, cv2.COLOR_BGR2RGB)

# Define the red color range in RGB
lower_red = np.array([100, 0, 0])
upper_red = np.array([255, 150, 150])

# Create a mask for red areas
red_mask = cv2.inRange(nrg_img_rgb, lower_red, upper_red)

# Apply the mask to the LIDAR data
lidar_no_red = np.where(red_mask_resized == 255, np.nan, lidar_data)

fig, axes = plt.subplots(1, 2, figsize=(15, 10))

# Plot the LIDAR data
axes[0].imshow(lidar_data, cmap='plasma')
axes[0].set_title('Original LIDAR Data')
axes[0].axis('off')

# Plot the LIDAR data with red areas removed
axes[1].imshow(lidar_no_red, cmap='plasma')
axes[1].set_title('LIDAR Data with Red Areas Removed')
axes[1].axis('off')

plt.tight_layout()
plt.show()

In [ ]:
fig, axes = plt.subplots(2, 2, figsize=(20, 20))

# Plot the original image
axes[0, 0].imshow(img, extent=[0, img.shape[1], 0, img.shape[0]])
axes[0, 0].set_title('Original Image')
axes[0, 0].axis('off')

# Overlay the LIDAR data with transparency
axes[0, 1].imshow(img, extent=[0, img.shape[1], 0, img.shape[0]])
axes[0, 1].imshow(lidar_data, cmap='plasma', alpha=0.6, extent=[0, img.shape[1], 0, img.shape[0]])
axes[0, 1].set_title('LIDAR Data Overlay on Original Image')
axes[0, 1].axis('off')

# Overlay the LIDAR data with red areas removed
axes[1, 0].imshow(img, extent=[0, img.shape[1], 0, img.shape[0]])
axes[1, 0].imshow(lidar_no_red, cmap='plasma', alpha=0.6, extent=[0, img.shape[1], 0, img.shape[0]])
axes[1, 0].set_title('LIDAR Data with Red Areas Removed Overlay on Original Image')
axes[1, 0].axis('off')

# Overlay the above ground LIDAR data with red areas removed
axes[1, 1].imshow(img, extent=[0, img.shape[1], 0, img.shape[0]])
axes[1, 1].imshow(above_ground_lidar_no_red, cmap='plasma', alpha=0.6, extent=[0, img.shape[1], 0, img.shape[0]])
axes[1, 1].set_title('Above Ground LIDAR Data with Red Areas Removed Overlay on Original Image')
axes[1, 1].axis('off')

plt.tight_layout()
plt.show()


In [ ]:
fig, ax = plt.subplots(figsize=(10, 10))

# Plot the original image
ax.imshow(img, extent=[0, img.shape[1], 0, img.shape[0]])

# Overlay the above ground LIDAR data with red areas removed
ax.imshow(above_ground_lidar_no_red, cmap='plasma', alpha=0.6, extent=[0, img.shape[1], 0, img.shape[0]])

ax.set_title('Above Ground LIDAR Data with Red Areas Removed Overlay on Original Image')
ax.axis('off')

plt.tight_layout()
plt.show()

In [ ]:
def calculate_average_height(lidar_data, n):
    grid_size_x = lidar_data.shape[1] // n
    grid_size_y = lidar_data.shape[0] // n

    average_heights = np.zeros((n, n))

    for i in range(n):
        for j in range(n):
            grid_data = lidar_data[i * grid_size_y:(i + 1) * grid_size_y, j * grid_size_x:(j + 1) * grid_size_x]
            average_heights[i, j] = np.nanmean(grid_data)

    return average_heights

n = 1000  # Define the grid size
average_heights = calculate_average_height(lidar_data, n)

# Plot the average heights
plt.imshow(average_heights, cmap='viridis')
plt.colorbar(label='Average Height')
plt.title(f'Average Heights in {n}x{n} Grid')
plt.show()

In [ ]:
threshold = 30
fig, ax = plt.subplots(figsize=(10, 10))

# Calculate the grid size
grid_size_x = lidar_data.shape[1] // n
grid_size_y = lidar_data.shape[0] // n

# Create a mask for points higher than the average elevation at that point
higher_than_average_mask = np.zeros_like(lidar_data, dtype=bool)

for i in range(n):
    for j in range(n):
        grid_data = lidar_data[i * grid_size_y:(i + 1) * grid_size_y, j * grid_size_x:(j + 1) * grid_size_x]
        higher_than_average_mask[i * grid_size_y:(i + 1) * grid_size_y, j * grid_size_x:(j + 1) * grid_size_x] = grid_data > average_heights[i, j]

# Apply the threshold and the higher than average mask
house_regions = (lidar_data > threshold) & higher_than_average_mask
num_labels, labels_im = cv2.connectedComponents(house_regions.astype(np.uint8))

ax.imshow(labels_im, cmap='nipy_spectral')
ax.set_title(f'Threshold: {threshold}\nHouses: {num_labels - 1}')
ax.axis('off')

plt.tight_layout()
plt.show()

In [ ]:
threshold_values = list(range(20, 40))
fig, axes = plt.subplots(4, 5, figsize=(20, 20))

for ax, threshold in zip(axes.flatten(), threshold_values):
    # Create a mask for points higher than the average elevation at that point
    higher_than_average_mask = np.zeros_like(lidar_data, dtype=bool)

    for i in range(n):
        for j in range(n):
            grid_data = lidar_data[i * grid_size_y:(i + 1) * grid_size_y, j * grid_size_x:(j + 1) * grid_size_x]
            higher_than_average_mask[i * grid_size_y:(i + 1) * grid_size_y, j * grid_size_x:(j + 1) * grid_size_x] = grid_data > average_heights[i, j]

    # Apply the threshold and the higher than average mask
    house_regions = (lidar_data > threshold) & higher_than_average_mask
    num_labels, labels_im = cv2.connectedComponents(house_regions.astype(np.uint8))

    ax.imshow(labels_im, cmap='nipy_spectral')
    ax.set_title(f'Threshold: {threshold}\nHouses: {num_labels - 1}')
    ax.axis('off')

plt.tight_layout()
plt.show()


In [ ]:
threshold = 30
min_size = 50  # Minimum size of objects to be considered as houses
max_size = 1000  # Maximum size of objects to be considered as houses

fig, ax = plt.subplots(figsize=(10, 10))

# Calculate the grid size
grid_size_x = lidar_data.shape[1] // n
grid_size_y = lidar_data.shape[0] // n

# Create a mask for points higher than the average elevation at that point
higher_than_average_mask = np.zeros_like(lidar_data, dtype=bool)

for i in range(n):
    for j in range(n):
        grid_data = lidar_data[i * grid_size_y:(i + 1) * grid_size_y, j * grid_size_x:(j + 1) * grid_size_x]
        higher_than_average_mask[i * grid_size_y:(i + 1) * grid_size_y, j * grid_size_x:(j + 1) * grid_size_x] = grid_data > average_heights[i, j]

# Apply the threshold and the higher than average mask
house_regions = (lidar_data > threshold) & higher_than_average_mask
num_labels, labels_im = cv2.connectedComponents(house_regions.astype(np.uint8))

# Filter out objects that are too small or too large
filtered_labels_im = np.zeros_like(labels_im)
for label in range(1, num_labels):
    label_mask = labels_im == label
    label_size = np.sum(label_mask)
    if min_size <= label_size <= max_size:
        filtered_labels_im[label_mask] = label

ax.imshow(filtered_labels_im, cmap='nipy_spectral')
ax.set_title(f'Threshold: {threshold}\nHouses: {np.max(filtered_labels_im)}')
ax.axis('off')

plt.tight_layout()
plt.show()

In [ ]:
fig, ax = plt.subplots(figsize=(10, 10))

# Plot the original image
ax.imshow(cv2.cvtColor(img, cv2.COLOR_BGR2RGB), extent=[0, img.shape[1], 0, img.shape[0]])

# Overlay the filtered LIDAR data with transparency
ax.imshow(filtered_labels_im, cmap='nipy_spectral', alpha=0.6, extent=[0, img.shape[1], 0, img.shape[0]])

ax.set_title('Filtered LIDAR Data Overlay on Original Image')
ax.axis('off')

plt.tight_layout()
plt.show()

In [ ]:
fig, ax = plt.subplots(figsize=(10, 10))

# Plot the original image
ax.imshow(cv2.cvtColor(img, cv2.COLOR_BGR2RGB), extent=[0, img.shape[1], 0, img.shape[0]])

# Overlay the house regions with transparency
ax.imshow(house_regions, cmap='nipy_spectral', alpha=0.6, extent=[0, img.shape[1], 0, img.shape[0]])

ax.set_title('House Regions Overlay on Original Image')
ax.axis('off')

plt.tight_layout()
plt.show()

In [ ]:
fig, ax = plt.subplots(figsize=(10, 10))

# Plot the original image
ax.imshow(cv2.cvtColor(img, cv2.COLOR_BGR2RGB), extent=[0, img.shape[1], 0, img.shape[0]])

# Overlay the filtered LIDAR data with red areas removed
ax.imshow(filtered_labels_im_no_red, cmap='viridis', alpha=0.6, extent=[0, img.shape[1], 0, img.shape[0]])

ax.set_title('Filtered LIDAR Data with Red Areas Removed Overlay on Original Image')
ax.axis('off')

plt.tight_layout()
plt.show()


In [ ]:
# Subtract the red areas (grassy areas) from the house regions
house_regions_no_red = house_regions & (red_mask_resized == 0)

# Plot the result
fig, ax = plt.subplots(figsize=(10, 10))

# Plot the original image
ax.imshow(cv2.cvtColor(img, cv2.COLOR_BGR2RGB), extent=[0, img.shape[1], 0, img.shape[0]])

# Overlay the house regions with red areas removed
ax.imshow(house_regions_no_red, cmap='nipy_spectral', alpha=0.6, extent=[0, img.shape[1], 0, img.shape[0]])

ax.set_title('House Regions with Red Areas Removed Overlay on Original Image')
ax.axis('off')

plt.tight_layout()
plt.show()

In [ ]:
threshold = 30
min_size = 50  # Minimum size of objects to be considered as houses
max_size = 1000  # Maximum size of objects to be considered as houses

fig, ax = plt.subplots(figsize=(10, 10))

# Calculate the grid size
grid_size_x = lidar_data.shape[1] // n
grid_size_y = lidar_data.shape[0] // n

# Create a mask for points higher than the average elevation at that point
higher_than_average_mask = np.zeros_like(lidar_data, dtype=bool)

for i in range(n):
    for j in range(n):
        grid_data = lidar_data[i * grid_size_y:(i + 1) * grid_size_y, j * grid_size_x:(j + 1) * grid_size_x]
        higher_than_average_mask[i * grid_size_y:(i + 1) * grid_size_y, j * grid_size_x:(j + 1) * grid_size_x] = grid_data > average_heights[i, j]

# Apply the threshold and the higher than average mask
house_regions = (lidar_data > threshold) & higher_than_average_mask
num_labels, labels_im = cv2.connectedComponents(house_regions.astype(np.uint8))

# Filter out objects that are too small or too large
filtered_labels_im = np.zeros_like(labels_im)
for label in range(1, num_labels):
    label_mask = labels_im == label
    label_size = np.sum(label_mask)
    if min_size <= label_size <= max_size:
        filtered_labels_im[label_mask] = label

# Subtract the red areas (grassy areas) from the house regions
house_regions_no_red = house_regions & (red_mask_resized == 0)

In [ ]:
threshold = 30
fig, ax = plt.subplots(figsize=(10, 10))

# Calculate the grid size
grid_size_x = lidar_data.shape[1] // n
grid_size_y = lidar_data.shape[0] // n

# Create a mask for points higher than the average elevation at that point
higher_than_average_mask = np.zeros_like(lidar_data, dtype=bool)

for i in range(n):
    for j in range(n):
        grid_data = lidar_data[i * grid_size_y:(i + 1) * grid_size_y, j * grid_size_x:(j + 1) * grid_size_x]
        higher_than_average_mask[i * grid_size_y:(i + 1) * grid_size_y, j * grid_size_x:(j + 1) * grid_size_x] = grid_data > average_heights[i, j]

# Apply the threshold and the higher than average mask
house_regions = (lidar_data > threshold) & higher_than_average_mask
num_labels, labels_im = cv2.connectedComponents(house_regions.astype(np.uint8))

ax.imshow(labels_im, cmap='nipy_spectral')
ax.set_title(f'Threshold: {threshold}\nHouses: {num_labels - 1}')
ax.axis('off')

plt.tight_layout()
plt.show()